<a href="https://colab.research.google.com/github/enerdb/cats_and_dogs/blob/main/Copy_of_cats_and_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip


--2021-09-20 18:01:08--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.193.128, 172.217.204.128, 172.217.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M   150MB/s    in 0.4s    

2021-09-20 18:01:09 (150 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
!unzip cats_and_dogs_filtered.zip

In [ ]:
!rm -rf cats_and_dogs_filtered.zip

In [ ]:
!pip install tensorflow

In [ ]:
# os auxilia em tarefes no Sistema operacional
import os
# plotar imagens
import matplotlib.pyplot as plt
# implementação do modelo
import tensorflow as tf


In [ ]:
dataset_dir = os.path.join(os.getcwd(), 'cats_and_dogs_filtered')

dataset_train_dir = os.path.join(dataset_dir, 'train')
dataset_train_cats_len = len(os.listdir(os.path.join(dataset_train_dir, 'cats')))
dataset_train_dogs_len = len(os.listdir(os.path.join(dataset_train_dir, 'dogs')))

dataset_validation_dir = os.path.join(dataset_dir, 'validation')
dataset_validation_cats_len = len(os.listdir(os.path.join(dataset_validation_dir, 'cats')))
dataset_validation_dogs_len = len(os.listdir(os.path.join(dataset_validation_dir, 'dogs')))

print('Train Cats: %s' % dataset_train_cats_len)
print('Train Dogs: %s' % dataset_train_dogs_len)
print('Validation Cats: %s' % dataset_validation_cats_len)
print('Validation Dogs: %s' % dataset_validation_dogs_len)

Train Cats: 1000
Train Dogs: 1000
Validation Cats: 500
Validation Dogs: 500


Preprocessing:

Size: 160x160 (scaling)
Value by color 0-255 RGB


In [ ]:
image_width = 160
image_height = 160
image_color_channel = 3
image_color_channel_size = 255
image_size = (image_width, image_height)
image_shape = image_size + (image_color_channel, )

# number of features used per batch
batch_size = 32 
# number of times the dataset will participate n training
epochs = 20
# learning rate
learning_rate = 0.0001

class_names = ['cat', 'dog']

In [ ]:
dataset_train = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_train_dir,
    image_size = image_size,
    batch_size = batch_size,
    shuffle = True
)

Found 2000 files belonging to 2 classes.


In [ ]:
dataset_validation_pre = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_validation_dir,
    image_size = image_size,
    batch_size = batch_size,
    shuffle = True
)

Found 1000 files belonging to 2 classes.


In [ ]:
dataset_validation_cardinality = tf.data.experimental.cardinality(dataset_validation_pre)
dataset_validation_batches = dataset_validation_cardinality // 5

dataset_test = dataset_validation_pre.take(dataset_validation_batches)
dataset_validation = dataset_validation_pre.skip(dataset_validation_batches)

print('Validation Dataset Cardinality: %d' % tf.data.experimental.cardinality(dataset_validation))
print('Test Dataset Cardinality: %d' % tf.data.experimental.cardinality(dataset_test))



Validation Dataset Cardinality: 26
Test Dataset Cardinality: 6


In [ ]:
# Function to display images of a dataset

def plot_dataset(dataset):

  plt.gcf().clear()
  plt.figure(figsize = (15,15))

  for features, labels in dataset.take(1):
      for i in range(9):
    
          plt.subplot(3, 3, i+1)
          plt.axis('off')

          plt.imshow(features[i].numpy().astype('uint8'))
          plt.title(class_names[labels[i]])


In [ ]:
# model training

# Sequential - each layer is applied after the other
model = tf.keras.models.Sequential([
    # Rescaling (160, 160, 3), values from 0 to 1
    tf.keras.layers.experimental.preprocessing.Rescaling(
        1. /image_color_channel_size,
        input_shape = image_shape
    ),
    # Convolutional Layer, Kernel 3x3 passed 16x
    # padding = same - zeros around feature
    # activation function = relu - zero if negative
    tf.keras.layers.Conv2D(16, 3, padding = 'same', activation = 'relu'),
    # MaxPooling consolidates the activation map from a Convolution layer
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D(),
    # Flattens the output
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(1, activation= 'sigmoid')
])

#

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = ['accuracy']
)

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 160, 160, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 160, 160, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 80, 80, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 80, 80, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 40, 40, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 40, 40, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 20, 20, 64)        0

In [ ]:
history = model.fit(
    dataset_train,
    validation_data = dataset_validation,
    epochs = epochs
)

Epoch 1/20
63/63 [==============================] - 54s 842ms/step - loss: 0.6911 - accuracy: 0.5295 - val_loss: 0.6692 - val_accuracy: 0.6101
Epoch 2/20
63/63 [==============================] - 53s 844ms/step - loss: 0.6598 - accuracy: 0.6115 - val_loss: 0.6372 - val_accuracy: 0.6671
Epoch 3/20
63/63 [==============================] - 54s 850ms/step - loss: 0.6175 - accuracy: 0.6640 - val_loss: 0.6177 - val_accuracy: 0.6844
Epoch 4/20
63/63 [==============================] - 54s 855ms/step - loss: 0.5835 - accuracy: 0.6950 - val_loss: 0.5928 - val_accuracy: 0.6757
Epoch 5/20
63/63 [==============================] - 54s 849ms/step - loss: 0.5465 - accuracy: 0.7390 - val_loss: 0.5832 - val_accuracy: 0.7067
Epoch 6/20
63/63 [==============================] - 53s 846ms/step - loss: 0.5191 - accuracy: 0.7565 - val_loss: 0.5653 - val_accuracy: 0.7042
Epoch 7/20
63/63 [==============================] - 53s 844ms/step - loss: 0.4962 - accuracy: 0.7665 - val_loss: 0.5627 - val_accuracy: 0.7079

In [ ]:
def plot_model():

  accuracy = history.history['accuracy'']
  val_accuracy = history.history['val_accuracy']

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs_range = range(epochs)

  plt.gcf().clear()
  plt.figure(figsize = (15,8))

  plt.subplot(1, 2, 1)
  plt.title('Training and Validation Accuracy')
  plt.plot(epochs_range, accuracy, label = 'Training Accuracy')
  plt.plot(epochs_range, val_accuracy, label = 'Validation Accuracy')
  plt.legend(loc = 'lower right')

  plt.subplot(1, 2, 2)
  plt.title('Training and Validation Loss')
  plt.plot(epochs_range, loss, label = 'Training Loss')
  plt.plot(epochs_range, val_loss, label = 'Validation Loss')
  plt.legend(loc = 'lower right')

  plt.show()

In [ ]:
plot_model()

In [ ]:
def plot_dataset_predictions(dataset):

  features, labels = dataset.as_numpy_iterator().next()

  predictions_aux = model.predict_on_batch(features).flatten()
  predictions = tf.where(predictions_flatten <0.5, 0, 1)

  print('Labels:      %s' % labels)
  print('Predictions: %s' % predictions.numpy())

  plt.gcf().clear()
  plt.figure(figsize = (15,15))

  for i in range(9):

    plt.subplot(3,3, i + 1)
    plt.axis('off')

    plt.imshow(features[i].astype('uint8'))
    plt.title(class_names[predictions[i]])


In [ ]:
plot_dataset_predictions(dataset_test)

In [ ]:
# save model for production
model.save('path/to/model')

In [ ]:
# load saved model
# model = tf.keras.load_model('path/to')